In [13]:
import os
import sys
sys.path.append('../../sharechat-scraper') # to access env variables
import pymongo
from pymongo import MongoClient
from s3_mongo_helper import initialize_mongo
from db_config import sharechat_db_config, simplesearch_db_config
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
import logging
from bson import ObjectId

In [3]:
source_coll = initialize_mongo(sharechat_db_config)

In [ ]:
start = datetime(2020, 10, 5)
end = datetime(2020, 10, 12)

print(source_coll.count_documents({"scraped_date": {'$gte':start,'$lt':end},
                    "media_type": {"$in": ["image", "video"]}}))

cursor = source_coll.find({"scraped_date": {'$gte':start,'$lt':end},
                    "media_type": {"$in": ["image", "video"]}})

In [ ]:
ss_coll = initialize_mongo(simplesearch_db_config)

In [ ]:
data = {}
# For each post within the date range in the source db 
for i in cursor:
    try:
        # Get the corresponding media hash from the simplesearch db
        ss_hash = ss_coll.find_one({"source_id": str(i["_id"])})["hash"]
        data[str(i["_id"])] = ss_hash
    except Exception:
        print(logging.traceback.format_exc())
        pass

In [ ]:
len(data)

In [ ]:
# Transform data and cluster identical media hashes
df = pd.DataFrame.from_dict(data, orient="index").reset_index().rename(columns={"index":"source_id", 0:"hash"})
df["count"] = ""
grouped = df.groupby(by="hash").agg({"count":"size", "source_id": list})
sorted = grouped.sort_values("count", ascending=False)
sorted.reset_index(inplace=True)
# Narrow down to clusters that contain atleast two posts
clusters = sorted[sorted["count"] >= 2]
clusters.reset_index(inplace=True, drop = True)
clusters

In [100]:
# Define function to return media links of clustered posts
def get_clustered_posts(ids):
    links = []
    for i in ids:
        links.append(source_coll.find_one({"_id":ObjectId(i)})["s3_url"])
    return links

In [ ]:
clusters["s3_url"] = clusters["source_id"].map(get_clustered_posts)